In [1]:
import numpy as np
import pandas as pd

In [64]:
# read data
df = pd.read_excel('../dataset/annotated_adaptations.xlsx')

print(f"#Cases: {len(set(df['PostId']))}")

def calculate_occurrence(arr):
    res = []
    for ele in arr:
        if ele == '/':
            res.append(1)
        else:
            res.append(int(ele))
    return res

# adjust the formats
df['ReuseGranularity'] = df['ReuseGranularity'].str.strip()
#df['Type'] = df['Type'].fillna('/')
df['TargetElement'] = df['TargetElement'].str.strip()
df['ModificationGranularity'] = df['ModificationGranularity'].fillna('/')
df['Position'] = df['Position'].fillna('/')
df['Position'] = df['Position'].str.strip()
df['#Occurrence'] = df['#Occurrence'].fillna('/')
df['IsContextBased'] = df['IsContextBased'].fillna(0)
df['IsContextBased'] = df['IsContextBased'].map(lambda x:int(x))
# df['RelationBetweenAdaptations'] = df['Relations'].fillna('/')
df['Locate'] = df['Locate'].fillna('/')
df['Distance'] = df['Distance'].fillna('/')
df['Distance'] = df['Distance'].map(lambda x:str(x))
df['Distance'] = df['Distance'].str.strip()
df['occ'] = calculate_occurrence(df['#Occurrence'])
df['VariableType'] = df['VariableType'].fillna(0)
df['VariableType'] = df['VariableType'].map(lambda x:int(x))

#Cases: 300


# RQ1

In [65]:
REUSE_GRAN = ['Method', 'Block', 'Statement']
gran_count = dict()
clean_count = dict()
cb_count = dict()
for rg in REUSE_GRAN:
    gran_count[rg] = 0
    clean_count[rg] = 0
    cb_count[rg] = 0

cnts = []
for postid, idx in df.groupby(['PostId']).groups.items():

    # count number of cases in different reuse granularity
    rg = df.iloc[idx[0]]['ReuseGranularity']
    gran_count[rg] += 1
    
    # count clean changes
    if int(df.iloc[idx[0]]['Index']) == 0:
        clean_count[rg] += 1
        cnts.append([postid,rg]+[0]*10)
        continue
        
    # count context-based cases
    if any(df.iloc[idx]['IsContextBased'] == 1):
        cb_count[rg] += 1
       
print(f'Reuse Granularity Distribution: {gran_count}')
print(f'Clean Counts: {clean_count}')
print(f'Context-based Counts: {cb_count}')

Reuse Granularity Distribution: {'Method': 133, 'Block': 101, 'Statement': 66}
Clean Counts: {'Method': 18, 'Block': 8, 'Statement': 10}
Context-based Counts: {'Method': 34, 'Block': 72, 'Statement': 42}


# RQ2

In [66]:
element_map = {
    # method-level
    'Method': 'MethodDeclaration',
    # block-level
    'If Block': 'Conditional',
    'Else Block': 'Conditional',
    'For Block': 'Loop',
    'While Block': 'Loop',
    'Switch Block': 'Conditional',
    'Case Block': 'Conditional',
    'Try Block': 'ExceptionHandling',
    'Catch Block': 'ExceptionHandling',
    'Declaration Block': 'VariableDeclaration',
    'Functional Block': 'MethodInvocation',
    # statement-level
    'Attribute Declaration': 'FieldDeclaration',
    'Method Annotation': 'MethodAnnotation',
    'Exception Declaration': 'Throws',
    'Parameter Annotation': 'ParameterAnnotation',
    'Parameter Declaration': 'ParameterDeclaration',
    'If Statement': 'ConditionalExpression',
    'Loop Condition': 'LoopExpression',
    'Assignment': 'Assignment',
    'Expression': 'Expression', # TODO: position
    'Method Invocation': 'MethodInvocation',
    'Instance Creation': 'InstanceCreation',
    'Variable Annotation': 'VariableAnnotation',
    'Variable Declaration': 'VariableDeclaration',
    'Return Statement': 'ReturnStatement',
    'Throw Exception': 'ThrowStatement',
    # token-level
    'Class Modifier': 'ClassModifier',
    'Class Name': 'ClassName',
    'Attribute Modifier': 'FieldModifier',
    'Attribute Name': 'FieldName',
    'Method Modifier': 'MethodModifier',
    'Method Name': 'MethodName',
    'Return Type': 'ReturnType',
    'Parameter Modifier': 'ParameterModifier',
    'Parameter Name': 'ParameterName',
    'Parameter Type': 'ParameterType',
    'Exception Type': 'ExceptionType',
    'Exception Modifier': 'ExceptionModifier',
    'Variable Modifier': 'VariableModifier',
    'Variable Type': 'VariableType',
    'Local Variable Name': 'VariableName',
    'Local Variable': 'Variable',
    'Cast Type': 'CastExpression',
    'Instance Type': 'InstanceType',
    'Exception Name': 'ExceptionName'
}

position_map = {
    'Instance Creation': 'InstanceCreation',
    'Catch Clause': 'ExceptionHandling',
    # expression
    'If Statement': 'ConditionalExpression',
    'Loop Condition': 'LoopExpression',
    # generic type
    'Class Declaration': 'ClassDeclaration',
    'Method Declaration': 'MethodSignature',
    'Variable Declaration': 'VariableDeclaration',
    # variable
    'Instance Creation Argument': 'InstanceArgument',
    'Invocation Argument': 'InvocationArgument',
    'Invocation Receiver': 'InvocationReceiver',
    'Assignment': 'Assignment',
    'Left Value in Assignment': 'Assignment',
    'Expression': 'Expression',
    'Loop Iterable': 'LoopIterable',
    # return
    'Return Statement': 'ReturnStatement'
}

In [62]:
# test the map
res = set()
for idx, row in df.iterrows():
    if row['ModificationGranularity'] == 'Token':
        if row['TargetElement'] not in element_map:
            res.add(row['TargetElement'])
print(res)

{'Attribute From Another Class', 'This Expression', 'String Literal', 'Generic Type', 'Array Access', 'Parameterized Type', 'Type Literal', 'Boolean Literal', 'Number Literal', 'Null Literal'}


In [67]:
def init_tree():
    tree = dict()
    tree['JavaFile'] = dict()
    tree['JavaFile']['count'] = 0
    tree['JavaFile']['Import'] = dict()
    tree['JavaFile']['Import']['count'] = 0
    tree['JavaFile']['Class'] = dict()
    tree['JavaFile']['Class']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['ClassModifier'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['ClassModifier']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['ClassAnnotation'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['ClassAnnotation']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['ClassName'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['ClassName']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['ParentClass'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['ParentClass']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['FieldDeclaration'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['FieldDeclaration']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['FieldDeclaration']['FieldAnnotation'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['FieldDeclaration']['FieldAnnotation']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['FieldDeclaration']['FieldModifier'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['FieldDeclaration']['FieldModifier']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['FieldDeclaration']['FieldType'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['FieldDeclaration']['FieldType']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['FieldDeclaration']['FieldName'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['FieldDeclaration']['FieldName']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['MethodAnnotation'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['MethodAnnotation']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['MethodModifier'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['MethodModifier']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['ReturnType'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['ReturnType']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['MethodName'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['MethodName']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['ParameterDeclaration'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['ParameterDeclaration']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['ParameterDeclaration']['ParameterType'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['ParameterDeclaration']['ParameterType']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['ParameterDeclaration']['ParameterName'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['ParameterDeclaration']['ParameterName']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['ParameterDeclaration']['ParameterAnnotation'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['ParameterDeclaration']['ParameterAnnotation']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['ParameterDeclaration']['ParameterModifier'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['ParameterDeclaration']['ParameterModifier']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['Throws'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['Throws']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['Throws']['ExceptionModifier'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['Throws']['ExceptionModifier']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['Throws']['ExceptionType'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodSignature']['Throws']['ExceptionType']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Block'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Block']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Block']['Conditional'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Block']['Conditional']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Block']['Conditional']['ConditionalExpression'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Block']['Conditional']['ConditionalExpression']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Block']['Loop'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Block']['Loop']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Block']['Loop']['LoopExpression'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Block']['Loop']['LoopExpression']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Block']['Loop']['LoopExpression']['LoopIterable'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Block']['Loop']['LoopExpression']['LoopIterable']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Block']['ExceptionHandling'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Block']['ExceptionHandling']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Block']['ExceptionHandling']['ExceptionName'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Block']['ExceptionHandling']['ExceptionName']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['MethodInvocation'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['MethodInvocation']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['MethodInvocation']['InvocationName'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['MethodInvocation']['InvocationName']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['MethodInvocation']['InvocationArgument'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['MethodInvocation']['InvocationArgument']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['MethodInvocation']['InvocationReceiver'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['MethodInvocation']['InvocationReceiver']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['InstanceCreation'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['InstanceCreation']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['InstanceCreation']['InstanceType'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['InstanceCreation']['InstanceType']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['InstanceCreation']['InstanceArgument'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['InstanceCreation']['InstanceArgument']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['VariableDeclaration'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['VariableDeclaration']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['VariableDeclaration']['VariableType'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['VariableDeclaration']['VariableType']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['VariableDeclaration']['VariableName'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['VariableDeclaration']['VariableName']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['VariableDeclaration']['VariableAnnotation'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['VariableDeclaration']['VariableAnnotation']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['VariableDeclaration']['VariableModifier'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['VariableDeclaration']['VariableModifier']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['Assignment'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['Assignment']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['Expression'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['Expression']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['CastExpression'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['CastExpression']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['ReturnStatement'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['ReturnStatement']['count'] = 0
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['ThrowStatement'] = dict()
    tree['JavaFile']['Class']['ClassDeclaration']['MethodDeclaration']['MethodBody']['Statement']['ThrowStatement']['count'] = 0
    return tree

def create_path_map(tree, path, path_map):
    """
    map a language construct to all its parents (in the path of program structure tree)

    :param tree: a dict to store the program structure tree
    :param path: a list to store the path during the tree traversal
    :param path_map: a dict to store the mapping from language construct to its parents
    :return: updated path_map
    """
    if len(tree) == 0:
        raise ValueError('Empty tree.')
    for key, value in tree.items():
        if key != 'count':
            path_map[key] = list(path) # create a copy of currently remembered path sequence
            cur_path = list(path)
            cur_path.append(key)
            path_map = create_path_map(tree[key], cur_path, path_map)
        # else:
        #     print(key, value)
    return path_map

def insert_node(tree, path, node_name):
    """
    insert a new node to the program structure tree

    :param tree: a dict to store the program structure tree
    :param path: a list to store the path during the tree traversal
    :param node_name: the name of the new node
    :return: updated tree
    """
    if tree == None:
        raise ValueError('Empty tree.')
    if len(path) == 0:
        raise ValueError('Empty path.')
    if len(path) == 1:
        tree[path[0]][node_name] = dict()
        tree[path[0]][node_name]['count'] = 0
    else:
        tree[path[0]] = insert_node(tree[path[0]], path[1:], node_name)
    return tree

def get_children(tree, path):
    """
    get the children of a language construct in the program structure tree

    :param tree: a dict to store the program structure tree
    :param path: a list to store the path during the tree traversal
    :return: a list of children
    """
    if tree == None:
        raise ValueError('Empty tree.')
    if len(path) == 0:
        raise ValueError('Empty path.')
    if len(path) == 1:
        return list(tree[path[0]].keys())
    else:
        return get_children(tree[path[0]], path[1:])

def update_count(tree, path, cnt):
    """
    update the count of a language construct in the program structure tree

    :param tree: a dict to store the program structure tree
    :param path: a list to store the path during the tree traversal
    :param cnt: the count to be added to the language construct
    :return: updated tree
    """
    if tree == None:
        raise ValueError('Empty tree.')
    if len(path) == 0:
        raise ValueError('Empty path.')
    if len(path) > 1:
        tree[path[0]] = update_count(tree[path[0]], path[1:], cnt)
    tree[path[0]]['count'] += cnt
    return tree

# summarize adaptations in the tree format
def summarize_to_tree(df):
    tree = init_tree()
    path_map = create_path_map(tree, list(), dict())
    for idx, row in df.iterrows():
        if row['Index'] == 0:
            continue
        ele = row['TargetElement']
        occ = row['#Occurrence']
        pos = row['Position']

        if ele == 'Comment':
            continue
        # rename labeled elements
        if ele in element_map.keys():
            ele = element_map[ele]
        if ele in path_map.keys():
            path = list(path_map[ele])
            path.append(ele)
        else:
            ele = ele.replace(' ', '')
            if pos in position_map.keys():
                pos = position_map[pos]
                if pos in path_map.keys():
                    path = list(path_map[pos])
                    path.append(pos)
                    children = get_children(tree, path)
                    if ele not in children:
                        tree = insert_node(tree, path, ele)
                    path.append(ele)
                else:
                    print(idx, ele, pos, 'not found in path_map')
                    continue
            else:
                print(idx, ele, pos, 'not found in position_map')
                continue
        tree = update_count(tree, path, occ)
    return tree

def print_tree(file_path, tree, threshold, depth=0):
    for key, value in tree.items():
        if key == 'count':
            continue
        if value['count'] > threshold:
            print('{}{} (Count: {})'.format(' ' * depth, key, value['count']))
            if file_path != None:
                with open(file_path, 'a') as f:
                    f.write('{}{} (Count: {})\n'.format(' ' * depth, key, value['count']))
        if isinstance(value, dict):
            print_tree(file_path, value, threshold, depth + 2)

t = summarize_to_tree(df)
print_tree(None, t, 10)

JavaFile (Count: 1384)
  Class (Count: 1384)
    ClassDeclaration (Count: 1384)
      FieldDeclaration (Count: 16)
      MethodDeclaration (Count: 1363)
        MethodSignature (Count: 236)
          MethodAnnotation (Count: 14)
          MethodModifier (Count: 64)
          MethodName (Count: 37)
          ParameterDeclaration (Count: 92)
            ParameterName (Count: 29)
          Throws (Count: 12)
        MethodBody (Count: 1116)
          Block (Count: 110)
            Conditional (Count: 65)
            Loop (Count: 15)
            ExceptionHandling (Count: 30)
          Statement (Count: 1006)
            MethodInvocation (Count: 459)
              InvocationArgument (Count: 133)
                Variable (Count: 53)
                StringLiteral (Count: 38)
                AttributeFromAnotherClass (Count: 22)
              InvocationReceiver (Count: 74)
                Variable (Count: 72)
            InstanceCreation (Count: 40)
              InstanceArgument (Count: 12)
 

In [ ]:
ca_df = df[df['IsContextBased']==1]
t = summarize_to_tree(ca_df)
print_tree(f'ca_full.txt', t, 0)
for s in list(['adaptive', 'corrective', 'perfective', 'preventive']):
    full_df = df[df['Manual'] == s]
    ca_df = df[(df['Manual'] == s) & (df['IsContextBased']==1)] # context-based
    t = summarize_to_tree(full_df)
    print_tree(f'adaptation_tree_{s}.txt', t, 0)
    print_tree(f'adaptation_tree_{s}_full.txt', t, 10)
    t = summarize_to_tree(ca_df)
    print_tree(f'adaptation_tree_{s}_context_based.txt', t, 0)
    print_tree(f'adaptation_tree_{s}_context_based_full.txt', t, 10)

# RQ3

In [58]:
header_list = ['Class Declaration','Attribute','Member Method','Member Class','Return Type','Method Name','Parameter',
               'Local Variable','Local Statement','Statement in Other Methods','Imported Package']
ctx_map = {'Attribute':'Global', 'Member Method':'Global', 'Class Declaration':'Global', 
           'Member Class':'Global', 'Anonymous Class':'Global',
           'Return Type':'Local', 'Method Name':'Local','Local Statement':'Local', 
           'Parameter':'Local', 'Local Variable':'Local', 'Method Declaration':'Local',
           'Statement in Other Methods':'Related', 
           'Imported Package':'External'}
ddf = pd.read_csv('./RQ3_dist_stats.csv')
ddf = ddf.fillna('/')
stat = {'Global':[],'Local':[],'Related':[],'External':[]}
for c in header_list:
    tmp = [int(x) for x in ddf[c] if x != '/']
    stat[ctx_map[c]] += tmp
    print(c, len(tmp))
    print(np.min(tmp),np.median(tmp),np.mean(tmp),np.max(tmp))
for k in stat.keys():
    print(k)
    tmp = np.array(stat[k])
    print(np.min(tmp),np.median(tmp),np.mean(tmp),np.max(tmp),np.quantile(tmp,0.95))

Class Declaration 14
1 79.0 148.64285714285714 499
Attribute 94
-65 29.5 58.297872340425535 438
Member Method 16
-47 -1.0 2.875 63
Member Class 4
-17 -2.5 -3.25 9
Return Type 25
1 1.0 3.44 25
Method Name 9
1 1.0 2.0 9
Parameter 69
1 2.0 3.101449275362319 20
Local Variable 59
1 7.0 10.40677966101695 76
Local Statement 24
1 2.0 2.9583333333333335 7
Statement in Other Methods 18
-15 12.5 38.5 306
Imported Package 20
2 26.0 58.05 386
Global
-65 25.0 59.328125 499 284.19999999999993
Local
1 2.0 5.39247311827957 76 19.75
Related
-15 12.5 38.5 306 198.8999999999998
External
2 26.0 58.05 386 236.8500000000001


# RQ4

In [69]:
# frequent itemset mining - Aprior Algorithm

cnt0 = 0
cnt1 = 0
# extract transactions (why, how)
trans = []
ca_df = df[df['IsContextBased']==1]
for idx, row in ca_df.iterrows():
    whys = row['ManualFinal']
    hows = []
    ctx = row['Context']
    occ = row['#Occurrence']
    if '/' in ctx:
        hows = ctx.split('/')
        cnt1 += occ
    else:
        hows.append(ctx)
        cnt0 += occ
    hows = list(set(hows))
    for h in hows:
        trans.append([idx, whys, h, occ])

arr = np.array(trans)
keys_why = np.unique(arr[:,1])
keys_how = np.unique(arr[:,2])
print(cnt0, cnt1)

275 47


In [ ]:
by_case = []
ca_df = df[df['IsContextBased']==1]
for idx, row in ca_df.iterrows():
    whys = row['ManualFinal']
    hows = []
    ctx = row['Context']
    occ = row['#Occurrence']
    if '/' in ctx:
        hows = ctx.split('/')
    else:
        hows.append(ctx)
    hows = list(set(hows))
    for h in hows:
        by_case.append([idx, whys, h, occ])

arr = np.array(by_case)
keys_1 = np.unique(arr[:,1])
keys_2 = np.unique(arr[:,2])

res = []

for why_key in keys_1:
    for how_key in keys_2:
        cnt = 0
        cnt_why = 0
        cnt_how = 0
        cnt_all = 0
        for c in by_case:
            if c[1] == why_key:
                cnt_why += c[3]
            if c[2] == how_key:
                cnt_how += c[3]
            if c[1] == why_key and c[2] == how_key:
                cnt += c[3]
            cnt_all += c[3]
        sup = cnt / cnt_all
        sup_why = cnt_why / cnt_all
        sup_how = cnt_how / cnt_all
        confidence = sup / sup_why
        lift = sup / (sup_why * sup_how)
        res.append([why_key, how_key, sup, confidence, lift])

df_confidence = pd.DataFrame(res, columns=['why', 'how', 'support', 'confidence', 'lift']).sort_values(by=['confidence'], ascending=False)
df_confidence.to_csv('RQ4_assoc_rule_sorted_by_confidence.csv', index=False)